# Programming

In [1]:
# TODO
#!{__import__('sys').executable} -m pip install --quiet packaging pandas

## Importing `pyenergyplus`

In [2]:
import contextlib
import builtins

@contextlib.contextmanager
def temporary_attr(o, name: str):
    a = builtins.getattr(o, name)
    try: yield
    finally: builtins.setattr(o, name, a)

import sys

@contextlib.contextmanager
def temporary_search_path(*paths):
    with temporary_attr(sys, 'path'):
        setattr(sys, 'path', [str(p) for p in paths])
        try: yield
        finally: pass

In [3]:
import os
import shutil

def find_energyplus():
    return os.path.dirname(
        os.path.realpath(shutil.which('energyplus'))
    )

In [4]:
# TODO
import os
os.environ['PATH'] += f''':{os.path.expanduser('~/.local/bin')}'''

energyplus_path = find_energyplus()
#energyplus_path = os.path.expanduser('~/.local/EnergyPlus-23-1-0')
with temporary_search_path(energyplus_path):
    import pyenergyplus as eplus
    import pyenergyplus.api

## `epems` - EnergyPlus API, Object-Oriented

<table style="text-align: center">
    <caption>Architecture Overview (*NIX)</caption>
    <tr>
        <th>EnergyPlus<br>OO Abstraction Layer</th>
        <th rowspan="2">&rlhar;</th>
        <th colspan="3">EnergyPlus<br>State Machine</th>
    </tr>
    <tr>
        <td>
            API<br>
            <code>epems</code>
        </td>
        <td>
            API<br>
            (<code>pyenergyplus</code>)<br>
            (<code>libenergyplusapi.so</code>)
        </td>
        <td>ABI<br>&rlhar;</td>
        <td>Kernel<br>(<code>energyplus</code>)</td>
    </tr>
</table>

In [5]:
import pandas as pd

# TODO
class _TODO_FactoryDataFrame:
    def __init__(self, factory):
        self._df_specs = None
        pass
    
    def __getitem__(self, k):
        
        pass

In [6]:
import typing

class Actuator:
    class Specs(typing.NamedTuple):
        component_type: str
        control_type: str
        actuator_key: str
        
    def __init__(
        self, 
        specs: Specs,
        # TODO
        _ep_api: eplus.api.EnergyPlusAPI, _ep_state
    ):
        self._specs = specs
        # TODO
        self._ep_api = _ep_api
        self._ep_state = _ep_state
        
    @property
    def _ep_handle(self):
        # TODO
        return self._ep_api.exchange.get_actuator_handle(
            self._ep_state,
            component_type=self._specs.component_type,
            control_type=self._specs.control_type,
            actuator_key=self._specs.actuator_key
        )
        
    @property
    def specs(self):
        return self._specs

    @property
    def value(self):
        return self._ep_api.exchange.get_actuator_value(
            self._ep_state,
            actuator_handle=self._ep_handle
        )

    @value.setter
    def value(self, n: float):
        self._ep_api.exchange.set_actuator_value(
            self._ep_state,
            actuator_handle=self._ep_handle,
            actuator_value=n
        )

    def reset(self):
        self._ep_api.exchange.reset_actuator(
            self._ep_state,
            actuator_handle=self._ep_handle
        )


In [7]:
class Variable:
    class Specs(typing.NamedTuple):
        variable_name: str
        variable_key: str

    def __init__(
        self, 
        specs: Specs,
        # TODO
        _ep_api: eplus.api.EnergyPlusAPI, _ep_state
    ):
        self._specs = specs
        # TODO
        self._ep_api = _ep_api
        self._ep_state = _ep_state

    @property
    def _ep_handle(self):
        return self._ep_api.exchange.get_variable_handle(
            self._ep_state,
            variable_name=self._specs.variable_name,
            variable_key=self._specs.variable_key
        )

    @property
    def specs(self):
        return self._specs

    @property
    def value(self):
        return self._ep_api.exchange.get_variable_value(
            self._ep_state,
            variable_handle=self._ep_handle
        )

In [8]:
import packaging
import io
import csv
import collections

import pandas as pd

import typing

class Environment:
    _ep_api = eplus.api.EnergyPlusAPI()
    assert (
        packaging.version.Version(_ep_api.api_version()) 
            >= packaging.version.Version('0.2')
    )

    def __init__(self):
        pass
    
    def __enter__(self):
        # TODO
        if getattr(self, '_ep_state', None) is not None:
            self._ep_api.state_manager.reset_state(self._ep_state)
        else:
            self._ep_state = self._ep_api.state_manager.new_state()
        return self
        
    def __exit__(self, *_exc_args):
        if getattr(self, '_ep_state', None) is None:
            raise Exception()
        self._ep_api.state_manager.delete_state(self._ep_state)

    def _exec(self, *args):
        return self._ep_api.runtime.run_energyplus(
            self._ep_state,
            command_line_args=args
        )

    @property
    def _ready(self):
        return self._ep_api.exchange.api_data_fully_ready(self._ep_state)

    @property
    def _available_data(self):
        # TODO NOTE headsup! upcoming version will include `get_api_data`
        with io.StringIO(
            self._ep_api.exchange
                .list_available_api_data_csv(self._ep_state)
                .decode()
        ) as f:
            def _ep_csv_reader(f, default_title=None):
                title = default_title
                for row in csv.reader(f):
                    if len(row) == 1:
                        title = row.pop()
                    yield title, row
        
            d = collections.defaultdict(lambda: [])
            for title, row in _ep_csv_reader(f):
                if not row:
                    continue
                d[title].append(row)
        
            colnames = {
                '**ACTUATORS**': ['type', 'component_type', 'control_type', 'actuator_key'],
                '**INTERNAL_VARIABLES**': ['type', 'variable_type', 'variable_key'],
                '**PLUGIN_GLOBAL_VARIABLES**': ['type', 'var_name'],
                '**TRENDS**': ['type', 'trend_var_name'],
                '**METERS**': ['type', 'meter_name'],
                '**VARIABLES**': ['type', 'variable_name', 'variable_key']
            }
 
            return {title: pd.DataFrame(d[title], columns=colnames[title]) for title in d}
    
    # TODO
    @property
    def actuator_specs(self):
        return self._available_data['**ACTUATORS**'][[
            'component_type', 
            'control_type', 
            'actuator_key'
        ]]

    # TODO dict-like selectors!
    # TODO .actuators.list()
    # TODO .actuators['<component_type>', '<control_type>', '<actuator_key>']
    def actuator(self, **specs):
        return Actuator(
            specs=Actuator.Specs(**specs), 
            _ep_api=self._ep_api, 
            _ep_state=self._ep_state
        )

    # TODO callbacks
    def _TODO_on(self, callable):
        # self._ep_api.runtime.callback_<...>(self._ep_state, callable)
        pass

### `epems` in Action

#### Setup

- (Context/State Management) Enter `epems` environment...
  Don't forget to `__close__` it when done \
  OR better yet, just use `python`'s `with`-statement:
  ```python
    with Environment() as e:
        # NOTE do whatever you need to do with `e`
        ...
  ```

In [9]:
ep_ems = Environment().__enter__()

- 🎉

In [10]:
ep_ems._exec('--help')


PythonLinkage: Linked to Python Version: "3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]"
Built on Platform: Ubuntu22.04_x86_64
Usage: energyplus [options] [input-file]
Options:
  -a, --annual                 Force annual simulation
  -c, --convert                Output IDF->epJSON or epJSON->IDF, dependent on
                               input file type
  -D, --design-day             Force design-day-only simulation
  -d, --output-directory ARG   Output directory path (default: current
                               directory)
  -h, --help                   Display help information
  -i, --idd ARG                Input data dictionary path (default: Energy+.idd
                               in executable directory)
  -j, --jobs ARG               Multi-thread with N threads; 1 thread with no
                               arg. (Currently only for G-Function generation)
  -m, --epmacro                Run EPMacro prior to simulation
  -p, --output-prefi

0

In [11]:
ep_ems = ep_ems.__enter__()

ep_ems._exec(
    # TODO
    '--design-day',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    #f'{energyplus_path}/ExampleFiles/CoolingTower_VariableSpeed_MultiCell.idf'
    f'{energyplus_path}/ExampleFiles/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
)

EnergyPlus Starting
EnergyPlus, Version 23.1.0-87ed9199d4, YMD=2023.09.13 20:18
Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combi

EnergyPlus Completed Successfully.


0

#### Actuators

In [12]:
df_avail_data = ep_ems._available_data

In [13]:
df_avail_data_act = df_avail_data.get('**ACTUATORS**')

- Component types of the actuators...

In [14]:
df_avail_data_act['component_type'].unique()

array(['Weather Data', 'Schedule:Compact', 'Schedule:Constant',
       'Material', 'People', 'Lights', 'ElectricEquipment', 'Surface',
       'Zone', 'Zone Infiltration', 'Plant Loop Overall',
       'Supply Side Half Loop', 'Demand Side Half Loop',
       'Demand Side Branch', 'Plant Component Pipe:Adiabatic',
       'Plant Component WaterUse:Connections', 'Supply Side Branch',
       'Plant Component Pump:ConstantSpeed',
       'Plant Component WaterHeater:Mixed',
       'Plant Component Coil:Heating:Water',
       'Plant Component Pump:VariableSpeed',
       'Plant Component Boiler:HotWater',
       'Plant Component HeatExchanger:FluidToFluid',
       'Plant Component Chiller:Electric:ASHRAE205',
       'Plant Component Coil:Cooling:Water',
       'Plant Component Coil:Cooling:WaterToAirHeatPump:EquationFit',
       'Plant Component Coil:Heating:WaterToAirHeatPump:EquationFit',
       'Plant Component FluidCooler:TwoSpeed',
       'Plant Component HeaderedPumps:VariableSpeed',
     

In [15]:
df_avail_data_act[df_avail_data_act['component_type'] == 'Plant Component Chiller:Electric:ASHRAE205']

,type,component_type,control_type,actuator_key
1424,Actuator,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER1;
1427,Actuator,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER2;


In [16]:
df_avail_data_act[df_avail_data_act['component_type'] == 'Plant Component CoolingTower:VariableSpeed']

,type,component_type,control_type,actuator_key
1504,Actuator,Plant Component CoolingTower:VariableSpeed,On/Off Supervisory,TOWERWATERSYS COOLTOWER 1;
1506,Actuator,Plant Component CoolingTower:VariableSpeed,On/Off Supervisory,TOWERWATERSYS COOLTOWER 2;


In [17]:
df_avail_data_act[
    df_avail_data_act['component_type'].isin(
        ('Zone Temperature Control', 'Zone Humidity Control')
    )
]

,type,component_type,control_type,actuator_key
1631,Actuator,Zone Temperature Control,Heating Setpoint,BASEMENT;
1632,Actuator,Zone Temperature Control,Cooling Setpoint,BASEMENT;
1633,Actuator,Zone Temperature Control,Heating Setpoint,CORE_BOTTOM;
1634,Actuator,Zone Temperature Control,Cooling Setpoint,CORE_BOTTOM;
1635,Actuator,Zone Temperature Control,Heating Setpoint,CORE_MID;
1636,Actuator,Zone Temperature Control,Cooling Setpoint,CORE_MID;
1637,Actuator,Zone Temperature Control,Heating Setpoint,CORE_TOP;
1638,Actuator,Zone Temperature Control,Cooling Setpoint,CORE_TOP;
1639,Actuator,Zone Temperature Control,Heating Setpoint,PERIMETER_BOT_ZN_3;
1640,Actuator,Zone Temperature Control,Cooling Setpoint,PERIMETER_BOT_ZN_3;


#### Simulation

In [18]:
class CallbackCounter:
    def __init__(self, fn=lambda *args, **kwargs: None):
        self._cnt = 0
        self._fn = fn

    def __call__(self, *args, **kwargs):
        self._cnt += 1
        return self._fn(*args, **kwargs)
    
    def summary(self):
        return f'{self!r} was called {self._cnt} times'

ep_ems = ep_ems.__enter__()

act = ep_ems.actuator(
    component_type='Zone Temperature Control',
    control_type='Cooling Setpoint',
    actuator_key='CORE_MID'
)

def demo_cb_advanced():
    # TODO
    while not ep_ems._ready:
        #print('not ready!')
        return

    demo_cb()

    act.value = 25.
    print(f'[act: {act.value}]', end=' ')
    

demo_cb = CallbackCounter()
# ep_ems._ep_api.runtime.callback_inside_system_iteration_loop
ep_ems._ep_api.runtime.callback_begin_zone_timestep_before_init_heat_balance(
    ep_ems._ep_state,
    lambda _: demo_cb_advanced()
)

# TODO callback_inside_system_iteration_loop?
'''
ep_ems._ep_api.runtime.callback_begin_new_environment(
    ep_ems._ep_state,
    #lambda _: print('callback_begin_new_environment')
    demo_cb
)
'''

ep_ems._exec(
    # TODO
    #'--design-day',
    #'--annual',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    f'{energyplus_path}/ExampleFiles/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
)

print(demo_cb.summary())

EnergyPlus Starting
EnergyPlus, Version 23.1.0-87ed9199d4, YMD=2023.09.13 20:18
Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combi

EnergyPlus Completed Successfully.


In [19]:
# TODO
_ = '''
ep_ems.actuator(
    component_type='Zone Temperature Control',
    control_type='Cooling Setpoint',
    actuator_key='CORE_MID'
)
'''

In [20]:
ep_ems.__exit__()